In [ ]:
############################################################################
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
############################################################################
#  DISCLAIMER
# The code shared by CEs with customers must be limited to non-production
# demo/example code or infrastructure automation scripts.
############################################################################
# Author : scavaletto@

# Vertex AI SDK for Python: applied examples **code-bison model**
<table align="left">
  <td>
    <a href=" ">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href=" ">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
<a href=" " target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Installing the package

In [17]:
!pip install google-cloud-aiplatform backoff --upgrade --quiet --user

! ^^^^ Do not forget to click the "Restart runtime" button above.

## Authenticating & Library

In [1]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [18]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [3]:
PROJECT_ID = "cloud-llm-preview1"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
BUCKET_NAME = "codefile-examples-bucket"  # @param {type:"string"}

PYTHON_FILE = "examplepythoncode.py" # @param {type:"string"}
ENDPOINT_ID = 'code-bison-001'# @param {type:"string"}

In [4]:
import io
import os
from google.cloud import storage

## Leer archivo desde Cloud Storage

Acceder al Cloud Storage y descargar archivo .py

In [5]:
# Authenticate to Google Cloud
storage_client = storage.Client()

# Get the bucket object
bucket = storage_client.bucket(BUCKET_NAME)

# Get the blob object
blob = bucket.blob(PYTHON_FILE)

# Download the blob to a local file
with io.open(PYTHON_FILE, "wb") as f:
  blob.download_to_file(f)

# Get the list of files in the /content folder
files = os.listdir("/content")

# Open the file
file = open(PYTHON_FILE, "r")
# Read the file contents
contents = file.read()

print(contents)


# -*- coding: utf-8 -*-
"""ExamplePythoncode.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1V1XjL5M0K6Z3vwYreVAUDL80ltcuMNCD?resourcekey=0-YvkFw76Hzpc5zlApen-uFA
"""

def is_numeric(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def main():
    print("(plain) Enter a value: ", end="")
    value = input()

    if is_numeric(value):
        print(value, "is numeric!")
    else:
        print(value, "is not numeric.")

    print()

    print("(right justify, zero fill) Enter another value: ", end="")
    value = input()

    value = value.rjust(10, "0")

    if is_numeric(value):
        print(value, "is numeric!")
    else:
        print(value, "is not numeric.")

    print()

    print("(trim) Enter a third value: ", end="")
    value = input()

    if is_numeric(value):
        print(value, "is numeric!")
    else:
        print(value, "is not numeric.")

i

Inicializar vertexai con modelos fundacionales - code-bison

In [8]:
import vertexai
from vertexai.preview.language_models import CodeGenerationModel
from vertexai.preview.language_models import CodeChatModel
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [9]:
code_generation_model = CodeGenerationModel.from_pretrained("code-bison@001")
code_completion_model = CodeGenerationModel.from_pretrained("code-gecko@001")
code_chat_model = CodeChatModel.from_pretrained("codechat-bison@001")

In [10]:
import backoff
from google.api_core.exceptions import ResourceExhausted

@backoff.on_exception(backoff.expo, ResourceExhausted, max_time=10)
def get_code_generation(code_generation_model,
                        prompt,
                        temperature: float = 0.5,
                        max_output_tokens: int = 256
) -> object:
    """Example of using Code Generation to write a function."""
    parameters = {
        "temperature": temperature,
        "max_output_tokens": max_output_tokens,
    }
    response = code_generation_model.predict(prefix=prompt,
                                             temperature=temperature,
                                             max_output_tokens=max_output_tokens)
    return response.text

@backoff.on_exception(backoff.expo, ResourceExhausted, max_time=10)
def get_code_completion(code_completion_model,
                        prompt,
                        temperature: float = 0.2,
                        max_output_tokens: int = 64
) -> object:
    """Example of using Code Generation to write a function."""
    parameters = {
        "temperature": temperature,
        "max_output_tokens": max_output_tokens,
    }
    response = code_completion_model.predict(prefix=prompt,
                                             temperature=temperature,
                                             max_output_tokens=max_output_tokens)
    return response.text

@backoff.on_exception(backoff.expo, ResourceExhausted, max_time=10)
def get_code_chat(code_chat_model,
                        prompt,
                        temperature: float = 0.2,
                        max_output_tokens: int = 1024
) -> object:
    """Example of using Code Generation to write a function."""
    parameters = {
        "temperature": temperature,
        "max_output_tokens": max_output_tokens,
    }
    chat = code_chat_model.start_chat() #initiate the session
    response = chat.send_message(prompt,
                                             temperature=temperature,
                                             max_output_tokens=max_output_tokens)
    return response.text

In [12]:
prompt = "Of this code:"+contents+ "Write the details of what this script is doing and show me in bullet points. Then give me recommendations for improvement. Response in Spanish"
print(get_code_chat(code_chat_model,prompt,
                          temperature=0.5, max_output_tokens=1024))

Este script hace lo siguiente:

* Define una función llamada `is_numeric()` que toma un valor como entrada y devuelve `True` si el valor es numérico o `False` si no lo es.
* La función `is_numeric()` utiliza la función incorporada `float()` para intentar convertir el valor de entrada en un número. Si la conversión es exitosa, la función devuelve `True`. Si la conversión falla, la función devuelve `False`.
* El script llama a la función `is_numeric()` tres veces. La primera vez, el valor de entrada es "123". La segunda vez, el valor de entrada es "123.45". La tercera vez, el valor de entrada es "abc".
* La función `is_numeric()` devuelve `True` cuando el valor de entrada es "123" o "123.45". La función devuelve `False` cuando el valor de entrada es "abc".

Aquí hay algunas recomendaciones para mejorar el script:

* El script podría ser más eficiente si utilizara la función incorporada `str.isdigit()` para determinar si un valor es numérico. La función `str.isdigit()` es más rápida que l

Inicializar vertexai con modelos fundacionales - text-bison

In [7]:
import vertexai
from vertexai.preview.language_models import TextGenerationModel, ChatModel

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [8]:
model = TextGenerationModel.from_pretrained("google/text-bison@001")

print(model.predict(
"Of this code:"+contents+ "Write the details of what this script is doing and show me in bullet points. Then give me recommendations for improvement. Response in Spanish",
        temperature=.2,
        max_output_tokens=1024,
        top_k=40,
        top_p=.95,
))

Este script hace lo siguiente:

* Define una función llamada `is_numeric` que recibe un valor como argumento y devuelve `True` si el valor es numérico, o `False` en caso contrario.
* Define una función llamada `main` que pide al usuario que introduzca un valor, lo pasa a la función `is_numeric` y muestra un mensaje en función del resultado.
* Repite el proceso anterior dos veces más, pero en la segunda ocasión el valor se justifica a la derecha y se rellena con ceros, y en la tercera ocasión el valor se recorta.

A continuación se ofrecen algunas recomendaciones para mejorar el script:

* Se podría utilizar la función `type` para comprobar si el valor es de tipo `float`.
* Se podría utilizar la función `str.isnumeric` para comprobar si el valor contiene solo caracteres numéricos.
* Se podría utilizar la función `re.match` para comprobar si el valor coincide con una expresión regular que represente un número.
